# StockPrice Predictor

In [1]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

### CUDA Compatibility

In [ ]:
if torch.cuda.is_available():
    print("Using GPU")
else:
    print("Using CPU")
print(tf.config.experimental.list_physical_devices())

### Load Dataset

In [ ]:
file_path = "../data/stock_data.xlsx"  # Replace with your Excel file name
sheet_name = "Sheet1"          # Replace with the correct sheet name if needed

# Read Excel file
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Ensure the date column is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Set the 'date' column as the index (optional but common for time series)
data.set_index('Date', inplace=True)

# Display the first few rows of the dataset
print(data.head())

In [ ]:
prices = data[['Close']].values

### Preprocess Data

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(prices)

# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 100  # Use 60 days of data to predict the next day
X, y = create_dataset(scaled_data, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)